In [5]:
#Object Tracking
import cv2
import numpy as np
from mtcnn import MTCNN
import time

detector = MTCNN()

In [10]:

cap = cv2.VideoCapture(0)
while True: 
    #Capture frame-by-frame
    __, frame = cap.read()
    
    #Use MTCNN to detect faces
    result = detector.detect_faces(frame)
    if result != []:
        for person in result:
            bounding_box = person['box']
            keypoints = person['keypoints']
            confidance = str(round(person['confidence'] * 100, 2))+"%"
            
            box_x,box_y=bounding_box[0], bounding_box[1]#top corner of box
            box_x1,box_y1=bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3]
            cv2.rectangle(frame, (box_x,box_y), (box_x1,box_y1), (0,155,255), 2)
    
            cv2.circle(frame,(keypoints['left_eye']), 2, (0,155,255), 2)
            cv2.circle(frame,(keypoints['right_eye']), 2, (0,155,255), 2)
            cv2.circle(frame,(keypoints['nose']), 2, (0,155,255), 2)
            cv2.circle(frame,(keypoints['mouth_left']), 2, (0,155,255), 2)
            cv2.circle(frame,(keypoints['mouth_right']), 2, (0,155,255), 2)
            cv2.putText(frame, confidance, (box_x1,box_y1), cv2.FONT_HERSHEY_COMPLEX, .5, (0,155,255), 1)

    #display resulting frame
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) &0xFF == ord('q'):
        break

#When everything's done, release capture
cap.release()
cv2.destroyAllWindows()

In [3]:
def face_align(img,leye,reye,desiredFaceWidth,desiredFaceHeight):
    #img_shape=(img.shape[:2])
    desiredLeftEye=(0.35, 0.35)
    #print(img_shape)
    print(leye,reye)
    dY = reye[1] - leye[1]
    dX = reye[0] - leye[0]
    angle = np.degrees(np.arctan2(dY, dX)) - 180
    print(angle)
    # compute the desired right eye x-coordinate based on the
    # desired x-coordinate of the left eye
    desiredRightEyeX = 1.0 - desiredLeftEye[0]
    
    # determine the scale of the new resulting image by taking
    # the ratio of the distance between eyes in the *current*
    # image to the ratio of distance between eyes in the
    # *desired* image
    
    
    dist = np.sqrt((dX ** 2) + (dY ** 2))
    desiredDist = (desiredRightEyeX - desiredLeftEye[0])
    desiredDist *= desiredFaceWidth
    scale = desiredDist / dist
    print(dist)
    print(scale)
    # compute center (x, y)-coordinates (i.e., the median point)
    # between the two eyes in the input image
    eyesCenter = ((leye[0] + reye[0]) // 2,
                  (leye[1] + reye[1]) // 2)
    cv2.circle(img,eyesCenter, 2, (0,155,255), 2)
    cv2.line(img, leye, reye, (0,155,255), 2)
    print(eyesCenter)
    # grab the rotation matrix for rotating and scaling the face
    M = cv2.getRotationMatrix2D(eyesCenter, angle, -scale)
    #M = cv2.getRotationMatrix2D(eyesCenter, angle, .5)
    # update the translation component of the matrix
    tX = desiredFaceWidth * 0.5
    tY = desiredFaceHeight * desiredLeftEye[1]
    M[0, 2] += (tX - eyesCenter[0])
    M[1, 2] += (tY - eyesCenter[1])
    
    # apply the affine transformation
    (w, h) = (desiredFaceWidth,desiredFaceHeight)
    output = cv2.warpAffine(img, M, (w, h),flags=cv2.INTER_CUBIC)
    #return the aligned face
    return output
    #return img

In [9]:
frame=cv2.imread('tst2.jpg')
start = time.time()
result = detector.detect_faces(frame)
if result != []:
        for person in result:
            bounding_box = person['box']
            keypoints = person['keypoints']
            confidance = str(round(person['confidence'] * 100, 2))+"%"
            
            box_x,box_y=bounding_box[0], bounding_box[1]#top corner of box
            box_x1,box_y1=bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3]
            #cv2.rectangle(frame, (box_x,box_y), (box_x1,box_y1), (0,155,255), 2)
            
            
            #face_align(img,leye,reye,nose,desiredFaceWidth,desiredFaceHeight)
            frame=face_align(frame,(keypoints['left_eye']),(keypoints['right_eye']),
                             300,300)
#             cv2.circle(frame,(keypoints['left_eye']), 2, (0,155,255), 2)
#             cv2.circle(frame,(keypoints['right_eye']), 2, (0,155,255), 2)
#             cv2.circle(frame,(keypoints['nose']), 2, (0,155,255), 2)
#             cv2.circle(frame,(keypoints['mouth_left']), 2, (0,155,255), 2)
#             cv2.circle(frame,(keypoints['mouth_right']), 2, (0,155,255), 2)
            #cv2.putText(frame, confidance, (box_x1,box_y1), cv2.FONT_HERSHEY_COMPLEX, .5, (0,155,255), 1)
            end = time.time()
            print(end - start)
#display resulting frame
cv2.imshow('frame',frame)
#if cv2.waitKey(1) &0xFF == ord('q'):
cv2.waitKey(0)
cv2.destroyAllWindows()

(319, 112) (401, 146)
-157.47943439710312
88.76936408468859
1.0138632953834992
(360, 129)
0.2648324966430664


In [ ]:
start = time.time()
print("hello")
end = time.time()
print(end - start)